In [ ]:
# ===== STEP 1: INSTALL REQUIRED PACKAGES =====
!pip install crewai==0.134.0
!pip install 'crewai[tools]'

(8.5.0)


In [ ]:
# ===== STEP 2: IMPORT LIBRARIES =====
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ===== STEP 3: SET UP API KEYS =====
from google.colab import userdata
import os

# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

# os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

NameError: name 'os' is not defined

In [ ]:
# ===== STEP 4: CHOOSE YOUR LLM =====
# 🚀 UNCOMMENT THE LLM YOU WANT TO USE:

groq_llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.1
)

gemini_llm = LLM(
    model="gemini/gemini-2.0-flash",
    # api_key=userdata.get('GEMINI_API_KEY'),
    temperature=0.1
)

# openai_llm = LLM(
#     model="openai/gpt-4o-mini",
#     temperature=0.1
# )

In [ ]:
# 💡 QUICK LLM GUIDE:
# 🟢 Groq = FREE + Super Fast (recommended for beginners!)
# 🟡 OpenAI = Premium Quality (costs money)
# 🟦 Gemini = Good Balance (cheap)

In [ ]:
dubexa_expert = Agent(
    role='DUBEXA Format Answer Expert',
    goal='Create comprehensive, well-structured answers in DUBEXA format that help candidates thoroughly understand and explain concepts',
    backstory="""You are a master educator and technical communication expert who specializes
    in the DUBEXA format. You excel at breaking down complex concepts into clear, structured
    explanations that follow the DUBEXA methodology:

    - D: Definition using key technical terms
    - U: Use case scenarios and real-world examples
    - B: Benefits of using the concept
    - E: Example code (when applicable)
    - X: Extra information, edge cases, and limitations
    - A: Analogies and metaphors for better understanding

    You make technical concepts accessible while maintaining accuracy and depth.""",
    tools=[],
    verbose=True,
    allow_delegation=False,
    max_iter=3,
    llm=gemini_llm
)

In [ ]:
dubexa_answer_task = Task(
        description=f"""Create comprehensive DUBEXA format answers for each of the technical questions about closures in Javascript.

        **DUBEXA Format Requirements:**

        **D - Definition:**
        - Provide a clear, technically accurate definition
        - Use proper technical terminology
        - Keep it concise but complete

        **U - Use Cases & Why We Need It:**
        - Explain real-world scenarios where this is used
        - Provide specific, practical examples
        - Explain WHY this concept/technology is important
        - Include industry context

        **B - Benefits:**
        - List clear advantages and benefits
        - Compare with alternatives when relevant
        - Explain what problems it solves

        **E - Example Code:**
        - Provide practical, working code examples
        - Include comments explaining key parts
        - Use modern, best-practice syntax
        - Make examples relevant to the concept

        **X - Extra Information:**
        - Cover edge cases and limitations
        - Mention common pitfalls or misconceptions
        - Include performance considerations
        - Discuss when NOT to use it

        **A - Analogies & Metaphors:**
        - Create relatable analogies that simplify the concept
        - Use everyday examples that non-technical people could understand
        - Make the analogy memorable and accurate

        **Instructions:**
        - Answer each question thoroughly in DUBEXA format
        - Ensure answers are interview-appropriate (not too long, but comprehensive)
        - Use proper formatting and structure
        - Include all 6 DUBEXA components for each question""",

        expected_output="""Complete DUBEXA format answers for the question, structured as:

        ## Question 1: [Question text]

        **Definition:**
        [Clear technical definition]

        **Use Cases & Why We Need It:**
        [Real-world examples and importance]

        **Benefits:**
        [Advantages and problem-solving capabilities]

        **Example Code:**
        ```[language]
        [Working code example with comments]
        ```

        **Extra Information:**
        [Edge cases, limitations, best practices]

        **Analogies & Metaphors:**
        [Relatable analogies for better understanding]

        ---

        [Repeat same format for Questions 2 and 3]

        Each answer should be comprehensive yet concise enough for interview delivery.""",

        agent=dubexa_expert
)

In [ ]:
interview_crew = Crew(
    agents=[dubexa_expert],
    tasks=[dubexa_answer_task],
    process=Process.sequential,
    verbose=True,
)

In [ ]:
result = interview_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1d0241c0-93d4-4411-aa54-f8353cfc0e50                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: DUBEXA Format Answer Expert                                                                             │
│                                                                                                                 │
│  Task: Create comprehensive DUBEXA format answers for each of the technical questions about closures in         │
│  Javascript.                                                                                                    │
│                                                                                                                 │
│          **DUBEXA Format Requirements:**                                                                        │
│                                                                                                                 │
│          **D - Definition:**                                                                                    │
│          - Provide a clear, technically accurate definition                                                     │
│          - Use proper technical terminology                                                                     │
│          - Keep it concise but complete                                                                         │
│                                                                                                                 │
│          **U - Use Cases & Why We Need It:**                                                                    │
│          - Explain real-world scenarios where this is used                                                      │
│          - Provide specific, practical examples                                                                 │
│          - Explain WHY this concept/technology is important                                                     │
│          - Include industry context                                                                             │
│                                                                                                                 │
│          **B - Benefits:**                                                                                      │
│          - List clear advantages and benefits                                                                   │
│          - Compare with alternatives when relevant                                                              │
│          - Explain what problems it solves                                                                      │
│                                                                                                                 │
│          **E - Example Code:**                                                                                  │
│          - Provide practical, working code examples                                                             │
│          - Include comments explaining key parts                                                                │
│          - Use modern, best-practice syntax                                                                     │
│          - Make examples relevant to the concept                                                                │
│                                                                                                                 │
│          **X - Extra Information:**                                                                             │
│          - Cover edge cases and limitations                                                                     │
│          - Mention common pitfalls or misconceptions                                                            │
│          - Include performance considerations          

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: DUBEXA Format Answer Expert                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Question 1: What is a closure in JavaScript?                                                                │
│                                                                                                                 │
│  **Definition:**                                                                                                │
│  A closure is a function's ability to remember and access its lexical scope, even when that function is         │
│  executed outside of that original scope. In simpler terms, a closure allows a function to access variables     │
│  from its surrounding scope after the outer function has finished executing. This is achieved because the       │
│  inner function retains a reference to the outer function's variables.                                          │
│                                                                                                                 │
│  **Use Cases & Why We Need It:**                                                                                │
│  Closures are fundamental in JavaScript for several reasons:                                                    │
│                                                                                                                 │
│  *   **Data Encapsulation and Privacy:** Closures allow you to create private variables and methods within a    │
│  function, preventing direct access from outside the function. This is crucial for creating modular and         │
│  maintainable code.                                                                                             │
│  *   **Event Handlers and Callbacks:** When attaching event listeners or using callbacks, closures are used to  │
│  maintain access to variables needed within the event handler or callback function.                             │
│  *   **Partial Application and Currying:** Closures enable you to create new functions by pre-filling some of   │
│  the arguments of an existing function.                                                                         │
│  *   **Module Pattern:** Closures are the backbone of the module pattern, which is used to create               │
│  self-contained, reusable components.                                                                           │
│                                                                                                                 │
│  **Benefits:**                                                                                                  │
│                                                                                                                 │
│  *   **Data Security:** Protects variables from unintended modification by external code.                       │
│  *   **Code Organization:** Promotes modularity and reduces global namespace pollution.                         │
│  *   **State Preservation:** Allows functions to maintain state across multiple invocations.                    │
│  *   **Flexibility:** Enables advanced programming techniques like currying and partial application.            │
│                                                                                                                 │
│  **Example Code:**                                                                                              │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5113dc1e-24dc-4054-a0ca-a7403566d0ad                                                                     │
│  Agent: DUBEXA Format Answer Expert                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1d0241c0-93d4-4411-aa54-f8353cfc0e50                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Question 1: What is a closure in JavaScript?                                                  │
│                                                                                                                 │
│  **Definition:**                                                                                                │
│  A closure is a function's ability to remember and access its lexical scope, even when that function is         │
│  executed outside of that original scope. In simpler terms, a closure allows a function to access variables     │
│  from its surrounding scope after the outer function has finished executing. This is achieved because the       │
│  inner function retains a reference to the outer function's variables.                                          │
│                                                                                                                 │
│  **Use Cases & Why We Need It:**                                                                                │
│  Closures are fundamental in JavaScript for several reasons:                                                    │
│                                                                                                                 │
│  *   **Data Encapsulation and Privacy:** Closures allow you to create private variables and methods within a    │
│  function, preventing direct access from outside the function. This is crucial for creating modular and         │
│  maintainable code.                                                                                             │
│  *   **Event Handlers and Callbacks:** When attaching event listeners or using callbacks, closures are used to  │
│  maintain access to variables needed within the event handler or callback function.                             │
│  *   **Partial Application and Currying:** Closures enable you to create new functions by pre-filling some of   │
│  the arguments of an existing function.                                                                         │
│  *   **Module Pattern:** Closures are the backbone of the module pattern, which is used to create               │
│  self-contained, reusable components.                                                                           │
│                                                                                                                 │
│  **Benefits:**                                                                                                  │
│                                                                                                                 │
│  *   **Data Security:** Protects variables from unintended modification by external code.                       │
│  *   **Code Organization:** Promotes modularity and reduces global namespace pollution.                         │
│  *   **State Preservation:** Allows functions to maintain state across multiple invocations.                    │
│  *   **Flexibility:** Enables advanced programming techniques like currying and partial application.            │
│                                                                                                                 │
│  **Example Code:**                                    